# Analysis of Mislabeled, swapped data


This jupyter notebook analyzes the trees obtained from mislabelled (aka swapped) sequences. The sequences are from the simulated dataset. Mislabaling a paralog is equivalent to getting two paralogs and swap them while keeping their label (aka gene/paralog-ID) untouched. This will generate as a consequence erroneus orthologs assignments. Since it is the simulated dataset ortholog assignment are known a priori and are ground trouth, to simulate a real world scenario wHere there is more confusion and mistakes about orthologs assignments, we decided to swap some sequences to create wrong ortholog sets. 
<br><br>
The swapping happens in two steps for each family in the simulated dataset (500): first select the species to do the swapping in, then among the paralogs in that species actually swap the desired number of sequences around. Both species selection and sequence selection is random. So the swapping combination (10, 9) refers to selcting 10 species in which 9 sequences will be swapped. The 9 sequences are swapped among themselves.


## Table of Contents
- [1. Chapter 1 - Introduction](#chapter-1-introduction)
- [2. Chapter 2 - R packages](#chapter-2-R-packages)
- [3. Chapter 3 - Input specification](#chapter-3-Input-specification)
- [4. Chapter 4 - Swapped label vs true topologies](#chapter-4-Swapped-label-vs-true-topologies)
- [5. Chapter 5 - RF tables report](#chapter-5-RF-tables-report)

<a id="chapter-1-introduction"></a>

## Chapter 1 - Introduction

TODO add description of notebook and how to use it.


<a id="chapter-2-R-packages"></a>

## Chapter 2 - R packages

Here is the list of packages needed for this notebook.

In [6]:
library("phangorn")
library("ggplot2")
library("reshape2")
library("geiger")
library("adephylo")
library("phytools")
library("rlist")
library("plotrix")
library("plyr")
library("dplyr")
library("hrbrthemes")
library("rstatix")
library("gplots")
library("viridis")
library('gtools')
library(gridExtra)
library(grid)
library("ggforce")
library("scales")
library("pROC")
library("tidyr")
library("stringr")
library("patchwork")
library("ggarchery")

<a id="chapter-3-Input-specification"></a>

## Chapter 3 - Input specification

Change the parent_of_all_dir variable to give the location of the input. All input files are expected to be under it. Otherwise change the single input location to you liking.<br>
All input files or directory paths are listed here.

In [7]:
# the location of the directory that contains all inputs for analyses 
parent_of_all_dir = "/users/cn/avignoli/1_paralogs/simulated_datasets/simulated_sequences/"

# The directory (one or more) that contain all the results from the main.nf pipeline run. It can be a glob path.
dir_containing_all_pipe_results = paste0(parent_of_all_dir, "rerun_25_species_corrected_branches/swapped_data/group*")

# The file containing the rename mapping of the reference paralog tree (generic)
reference_tree_codefile = file.path(parent_of_all_dir, "ref_tree.code")

# get the species/organism list of names used by the main.nf pipeline
species_name_file = file.path(parent_of_all_dir, "orthologs_org_ids_to_concatenate")


# each run of main.nf has one output directory. here we get that or multiple runs outputs directories.
run_dirs = Sys.glob(dir_containing_all_pipe_results)
run_dirs = run_dirs[file.info(run_dirs)$isdir]

# Create the 'figures' directory if it doesn't exist. Created where this notebook is executed.
if (!dir.exists("figures")) {
    dir.create("figures")
}

# Create the 'tables' directory if it doesn't exist. Created where this notebook is executed.
if (!dir.exists("tables")) {
    dir.create("tables")
}

<a id="chapter-4-Swapped-label-vs-true-topologies"></a>

## Chapter 4 - Swapped label vs true topologies

TODO add description 

In [3]:
# later on to this file at each family iteretion a df rows will be appended. to avoid that running this cell multiple times might append to the file over and over this line is here to write the column name as well as overwrite an existing file so it is fresh.
write.table(data.frame(method = character(), group = character(), family = character(), swap_species = character(), swap_seq = character(), rf_val = numeric()),  
            file = "tables/RF_swapped_set_all_methods_vs_reference.tsv",  
            quote = FALSE, sep = "\t", row.names = FALSE, col.names = TRUE)

for (run in run_dirs) {
    print(basename(run))

    # the mapping used later on to rename species trees according to a more generic sequence name
    ref_codefile = read.table(file = reference_tree_codefile )
    
    # get all the family subdirectories in each main.nf output run dir. 
    # This will contain the 100 normal families directories + all directories of mislabeled families.
    # There are (90) combination of 10 species and 10 sequences swaps (no swap on one sequence) per family, so 9100 dirs in each group.
    fam_dirs = grep("avg_branchlen_0.7_protein_family*", list.dirs(run, recursive = FALSE), value = TRUE)
    count_fam = -1

    # read the names of the ortholog species.
    orgs = read.table(file = species_name_file)

    # setting global df that will hold family level rf values.
    df_all_rf = data.frame(
        method       = c(),
        group        = c(),
        family       = c(),
        swap_species = c(),
        swap_seq     = c(),
        rf_val       = c()
    )

    # get all unit25 paralog trees for all combination of methods ME,ML + SM,ST
    for (m in fam_dirs) {
        
        count_fam = count_fam + 1

        # print every 10 families a check on progress to screen
        if (count_fam %% 910 == 0) {
            print(m)
        }

        # work only on swapped families directories
        if (grepl('swap_', m)) {

            # each directory has two subdirectories in which the data are stored
            tree_dir = paste0(m, "/results/rerun")

            # Read the paralog trees
            #Minimun Evolution (ME) + supermatrix method (SM)
            me_sm_path = list.files(path = tree_dir, pattern = glob2rx("*supermatrix.phylip_fastme_tree.nwk"), full.names = TRUE)
            me_sm_file = file.path(me_sm_path)
            me_sm_tree = read.tree(me_sm_file)
            me_sm_tree = unroot(me_sm_tree)

            # ME + supertree method (ST)
            me_st_path = list.files(path = tree_dir, pattern = glob2rx("avg_branchlen_*_paralog_tree.nwk"), full.names = TRUE)
            me_st_file = file.path(me_st_path)
            me_st_tree = read.tree(me_st_file)
            me_st_tree = unroot(me_st_tree)

            # Maximum Likelihood (ML) + SM
            ml_sm_path = list.files(path = tree_dir, pattern = glob2rx("RAxML_bestTree*_supermatrix_raxml.nwk"), full.names = TRUE)
            ml_sm_file = file.path(ml_sm_path)
            ml_sm_tree = read.tree(ml_sm_file)
            ml_sm_tree = unroot(ml_sm_tree)

            # ML + ST
            ml_st_path = list.files(path = tree_dir, pattern = glob2rx("RAxML_bestTree*_paralog_tree.nwk"), full.names = TRUE)
            ml_st_file = file.path(ml_st_path)
            ml_st_tree = read.tree(ml_st_file)
            ml_st_tree = unroot(ml_st_tree)

            # check if there any non-bynary (consensus trees) among the 4 trees present in the files above
            all_method_trees = c(me_sm_tree, me_st_tree, ml_sm_tree, ml_st_tree)
            if (length(which(is.binary(all_method_trees) == FALSE)) != 0) {
                
                # if there are non-binary trees, iterate through them and transformed into a binary one using multi2di package
                for (tree in which(is.binary(all_method_trees) == FALSE)) {
                    all_method_trees[[tree]] = multi2di(all_method_trees[[tree]])
                }
            }

            # read the reference tree associated with this family.
            ref_tree_file = file.path(paste0(strsplit(m, '_swap_')[[1]][1], '.nwk'))
            ref_tree      = read.tree(ref_tree_file)

            # Since each gene family has a predifined gene tree (ref_tree) composed of a fixed species tree pasted as leaf of the paralog tree. The tree is designed like this by choice. Each gene families varies only in the paralog tree that it has that is unique (15 tips).
            # Here we extract the paralog tree and to do so we just need to extract the subtree with all the sequences of omne species. Since each species have the same paralog tree. So we can extract the subtree of whichever species it does not matter.
            ref_subtree   = keep.tip(ref_tree, as.character(ref_codefile$V1))
    
            # we then rename the tips of the paralog tree (ref_subtree) to a more generic keyname. Effectively going from rat1 to seq1. 
            for (j in 1:length(ref_subtree$tip.label)) { 
                ref_subtree$tip.label[j] = as.character(ref_codefile$V2[which(ref_codefile$V1 == ref_subtree[["tip.label"]][j])])
            }
    
            # Unroot the reference tree 
            ref_subtree = unroot(ref_subtree)

            # compute the RF distance matrix, all against all. 5 tree in total.
            all_trees        = c(ref_subtree,
                                 all_method_trees
                                )
            class(all_trees) = "multiPhylo"
            all_vs_all_rf    = RF.dist(all_trees, normalize = TRUE)

            # update the family df with rf values. the first fopur columns together will act as an ID for tht tree.
            df_all_rf = data.frame(
                        method       = c(df_all_rf$method, c("SM_ME", "ST_ME", "SM_ML", "ST_ML")),
                        group        = c(df_all_rf$group, rep(basename(run), each = 4)),
                        family       = c(df_all_rf$family, rep(strsplit(basename(m), '_swap_')[[1]][1], each = 4)),
                        swap_species = c(df_all_rf$swap_species, rep(strsplit(m, "_")[[1]][length(strsplit(m, "_")[[1]])-1], each = 4)),      # getting the number of species affected by the swap
                        swap_seq     = c(df_all_rf$swap_seq, rep(strsplit(m, "_")[[1]][length(strsplit(m, "_")[[1]])], each = 4)),  # getting the number of sequences swapped per species
                        rf_val       = c(df_all_rf$rf_val,  c(all_vs_all_rf[1], all_vs_all_rf[2], all_vs_all_rf[3], all_vs_all_rf[4]))
            )

        # every time a non combination dir is encountered write to file what has been stored up untill now and reset the df
        } else {

            # appending the table with family rf values to the file. done this way so it is faster
            write.table(df_all_rf, file = "tables/RF_swapped_set_all_methods_vs_reference.tsv", quote = FALSE, sep = "\t", row.names = FALSE, col.names = FALSE, append = TRUE)

            # empty df
            df_all_rf = data.frame(
                method       = c(),
                group        = c(),
                family       = c(),
                swap_species = c(),
                swap_seq     = c(),
                rf_val       = c()
            )
        }
    }

    # appending the last family combinations of that group
    write.table(df_all_rf, file = "tables/RF_swapped_set_all_methods_vs_reference.tsv", quote = FALSE, sep = "\t", row.names = FALSE, col.names = FALSE, append = TRUE)

} 

[1] "group1_23th"
[1] "/users/cn/avignoli/1_paralogs/simulated_datasets/simulated_sequences/rerun_25_species_corrected_branches/swapped_data/group1_23th/avg_branchlen_0.7_protein_family_001.sub"
[1] "/users/cn/avignoli/1_paralogs/simulated_datasets/simulated_sequences/rerun_25_species_corrected_branches/swapped_data/group1_23th/avg_branchlen_0.7_protein_family_010.sub_swap_7_8"
[1] "/users/cn/avignoli/1_paralogs/simulated_datasets/simulated_sequences/rerun_25_species_corrected_branches/swapped_data/group1_23th/avg_branchlen_0.7_protein_family_020.sub_swap_5_6"
[1] "/users/cn/avignoli/1_paralogs/simulated_datasets/simulated_sequences/rerun_25_species_corrected_branches/swapped_data/group1_23th/avg_branchlen_0.7_protein_family_030.sub_swap_3_4"
[1] "/users/cn/avignoli/1_paralogs/simulated_datasets/simulated_sequences/rerun_25_species_corrected_branches/swapped_data/group1_23th/avg_branchlen_0.7_protein_family_040.sub_swap_10_4"
[1] "/users/cn/avignoli/1_paralogs/simulated_datasets/simula

<a id="chapter-5-RF-tables-report"></a>

## Chapter 5 - RF tables report

TODO add description 

In [26]:
# group 1, ME + SM 

# read the table compiled in chapter 4. So this cell does not depend on that one to run, it just needs the table file.
df_new = read.table("tables/RF_swapped_set_all_methods_vs_reference.tsv", header=T)

# filter the above table to retain only rf values associated with trees coming from group 1 and method ME + SM 
df_group1_ME_SM = df_new  %>%
  filter(group == "group1_23th", method == "SM_ME")

# Group by the swap_species and swap_seq per swap combination and calculate mean rf_val and sd
df_group1_ME_SM_processed = df_group1_ME_SM %>%
    group_by(swap_species, swap_seq) %>%
    summarise(
        mean_rf = round(mean(rf_val), 3),
        sd_rf = round(sd(rf_val), 3),
        .groups = "drop"
    ) %>%
    # Combine mean and sd into one string
    mutate(mean_sd = paste0(mean_rf, " ± ", sd_rf)) %>%
    select(swap_species, swap_seq, mean_sd) %>%
    # Create a wide format 10x10 matrix/table
    pivot_wider(names_from = swap_seq, values_from = mean_sd, values_fill = NA) %>%
    # Arrange rows in order
    arrange(swap_species)

df_group1_ME_SM_processed
write.table(df_group1_ME_SM_processed, file = "tables/avg_RF_swapped_set_group1_ME_SM_vs_reference.tsv", quote = FALSE, row.names = FALSE, sep = "\t")

swap_species,2,3,4,5,6,7,8,9,10,15
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,0.347 ± 0.15,0.341 ± 0.148,0.344 ± 0.152,0.34 ± 0.151,0.343 ± 0.155,0.331 ± 0.145,0.341 ± 0.15,0.335 ± 0.15,0.338 ± 0.163,NA
2,0.343 ± 0.147,0.346 ± 0.149,0.337 ± 0.146,0.326 ± 0.147,0.35 ± 0.151,0.337 ± 0.149,0.341 ± 0.143,0.338 ± 0.151,0.334 ± 0.15,NA
3,0.338 ± 0.152,0.346 ± 0.151,0.339 ± 0.148,0.348 ± 0.151,0.34 ± 0.15,0.344 ± 0.157,0.341 ± 0.156,0.339 ± 0.153,0.332 ± 0.15,NA
4,0.335 ± 0.152,0.345 ± 0.156,0.337 ± 0.149,0.334 ± 0.152,0.348 ± 0.154,0.342 ± 0.144,0.348 ± 0.167,0.327 ± 0.154,0.346 ± 0.159,NA
5,0.341 ± 0.146,0.343 ± 0.162,0.342 ± 0.154,0.34 ± 0.159,0.338 ± 0.156,0.341 ± 0.159,0.346 ± 0.147,0.328 ± 0.143,0.334 ± 0.16,NA
6,0.344 ± 0.161,0.337 ± 0.144,0.332 ± 0.148,0.339 ± 0.144,0.333 ± 0.152,0.345 ± 0.165,0.33 ± 0.153,0.329 ± 0.159,0.334 ± 0.16,NA
7,0.342 ± 0.148,0.333 ± 0.16,0.327 ± 0.138,0.322 ± 0.161,0.332 ± 0.151,0.333 ± 0.153,0.335 ± 0.144,0.342 ± 0.169,0.333 ± 0.154,NA
8,0.335 ± 0.146,0.333 ± 0.148,0.328 ± 0.147,0.347 ± 0.137,0.339 ± 0.151,0.335 ± 0.165,0.313 ± 0.162,0.338 ± 0.144,0.328 ± 0.162,NA
9,0.341 ± 0.155,0.34 ± 0.154,0.341 ± 0.149,0.346 ± 0.146,0.345 ± 0.148,0.331 ± 0.148,0.35 ± 0.165,0.344 ± 0.158,0.332 ± 0.148,NA


In [27]:
# group 1, ME + ST 

# read the table compiled in chapter 4. So this cell does not depend on that one to run, it just needs the table file.
df_new = read.table("tables/RF_swapped_set_all_methods_vs_reference.tsv", header=T)

# filter the above table to retain only rf values associated with trees coming from group 1 and method ME + ST 
df_group1_ME_ST = df_new  %>%
  filter(group == "group1_23th", method == "ST_ME")

# Group by the swap_species and swap_seq per swap combination and calculate mean rf_val and sd
df_group1_ME_ST_processed = df_group1_ME_ST %>%
    group_by(swap_species, swap_seq) %>%
    summarise(
        mean_rf = round(mean(rf_val), 3),
        sd_rf = round(sd(rf_val), 3),
        .groups = "drop"
    ) %>%
    # Combine mean and sd into one string
    mutate(mean_sd = paste0(mean_rf, " ± ", sd_rf)) %>%
    select(swap_species, swap_seq, mean_sd) %>%
    # Create a wide format 10x10 matrix/table
    pivot_wider(names_from = swap_seq, values_from = mean_sd, values_fill = NA) %>%
    # Arrange rows in order
    arrange(swap_species)


df_group1_ME_ST_processed
write.table(df_group1_ME_ST_processed, file = "tables/avg_RF_swapped_set_group1_ME_ST_vs_reference.tsv", quote = FALSE, row.names = FALSE, sep = "\t")

swap_species,2,3,4,5,6,7,8,9,10,15
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,0.356 ± 0.161,0.353 ± 0.163,0.349 ± 0.16,0.35 ± 0.161,0.352 ± 0.16,0.353 ± 0.159,0.348 ± 0.161,0.354 ± 0.163,0.349 ± 0.158,NA
2,0.351 ± 0.161,0.35 ± 0.163,0.349 ± 0.162,0.348 ± 0.158,0.354 ± 0.16,0.353 ± 0.158,0.352 ± 0.161,0.352 ± 0.159,0.354 ± 0.165,NA
3,0.352 ± 0.163,0.352 ± 0.16,0.356 ± 0.165,0.358 ± 0.159,0.356 ± 0.167,0.354 ± 0.157,0.35 ± 0.16,0.355 ± 0.161,0.348 ± 0.153,NA
4,0.354 ± 0.162,0.353 ± 0.158,0.353 ± 0.154,0.358 ± 0.158,0.352 ± 0.167,0.353 ± 0.166,0.351 ± 0.155,0.349 ± 0.163,0.353 ± 0.161,NA
5,0.354 ± 0.163,0.352 ± 0.158,0.356 ± 0.164,0.351 ± 0.164,0.35 ± 0.158,0.356 ± 0.168,0.353 ± 0.163,0.357 ± 0.159,0.355 ± 0.156,NA
6,0.358 ± 0.161,0.344 ± 0.149,0.351 ± 0.157,0.355 ± 0.162,0.357 ± 0.158,0.35 ± 0.158,0.358 ± 0.159,0.348 ± 0.162,0.349 ± 0.171,NA
7,0.357 ± 0.159,0.354 ± 0.163,0.352 ± 0.159,0.352 ± 0.16,0.353 ± 0.154,0.353 ± 0.166,0.355 ± 0.161,0.352 ± 0.162,0.358 ± 0.156,NA
8,0.354 ± 0.163,0.356 ± 0.165,0.354 ± 0.16,0.358 ± 0.163,0.343 ± 0.157,0.355 ± 0.163,0.344 ± 0.167,0.358 ± 0.153,0.348 ± 0.15,NA
9,0.349 ± 0.157,0.348 ± 0.16,0.357 ± 0.158,0.348 ± 0.156,0.353 ± 0.156,0.355 ± 0.161,0.356 ± 0.166,0.358 ± 0.16,0.351 ± 0.162,NA


In [29]:
# group 1, ML + SM 

# read the table compiled in chapter 4. So this cell does not depend on that one to run, it just needs the table file.
df_new = read.table("tables/RF_swapped_set_all_methods_vs_reference.tsv", header=T)

# filter the above table to retain only rf values associated with trees coming from group 1 and method ME + SM 
df_group1_ML_SM = df_new  %>%
  filter(group == "group1_23th", method == "SM_ML")

# Group by the swap_species and swap_seq per swap combination and calculate mean rf_val and sd
df_group1_ML_SM_processed = df_group1_ML_SM %>%
    group_by(swap_species, swap_seq) %>%
    summarise(
        mean_rf = round(mean(rf_val), 3),
        sd_rf = round(sd(rf_val), 3),
        .groups = "drop"
    ) %>%
    # Combine mean and sd into one string
    mutate(mean_sd = paste0(mean_rf, " ± ", sd_rf)) %>%
    select(swap_species, swap_seq, mean_sd) %>%
    # Create a wide format 10x10 matrix/table
    pivot_wider(names_from = swap_seq, values_from = mean_sd, values_fill = NA) %>%
    # Arrange rows in order
    arrange(swap_species)

df_group1_ML_SM_processed
write.table(df_group1_ML_SM_processed, file = "tables/avg_RF_swapped_set_group1_ML_SM_vs_reference.tsv", quote = FALSE, row.names = FALSE, sep = "\t")

swap_species,2,3,4,5,6,7,8,9,10,15
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,0.272 ± 0.155,0.271 ± 0.148,0.269 ± 0.148,0.277 ± 0.154,0.272 ± 0.149,0.277 ± 0.151,0.265 ± 0.154,0.282 ± 0.149,0.268 ± 0.15,NA
2,0.277 ± 0.154,0.276 ± 0.155,0.27 ± 0.158,0.268 ± 0.156,0.28 ± 0.162,0.262 ± 0.144,0.275 ± 0.147,0.272 ± 0.157,0.282 ± 0.153,NA
3,0.269 ± 0.162,0.269 ± 0.146,0.273 ± 0.156,0.284 ± 0.156,0.27 ± 0.147,0.287 ± 0.158,0.274 ± 0.149,0.269 ± 0.146,0.268 ± 0.144,NA
4,0.273 ± 0.154,0.277 ± 0.16,0.278 ± 0.154,0.278 ± 0.146,0.283 ± 0.148,0.271 ± 0.154,0.267 ± 0.154,0.264 ± 0.156,0.272 ± 0.147,NA
5,0.274 ± 0.15,0.272 ± 0.147,0.262 ± 0.153,0.269 ± 0.149,0.269 ± 0.145,0.277 ± 0.16,0.272 ± 0.15,0.27 ± 0.154,0.273 ± 0.153,NA
6,0.268 ± 0.142,0.285 ± 0.154,0.273 ± 0.136,0.284 ± 0.132,0.27 ± 0.145,0.273 ± 0.155,0.27 ± 0.15,0.28 ± 0.148,0.275 ± 0.146,NA
7,0.284 ± 0.159,0.268 ± 0.147,0.273 ± 0.152,0.258 ± 0.139,0.267 ± 0.148,0.287 ± 0.164,0.268 ± 0.143,0.285 ± 0.162,0.265 ± 0.142,NA
8,0.265 ± 0.149,0.274 ± 0.144,0.279 ± 0.153,0.283 ± 0.148,0.277 ± 0.147,0.276 ± 0.159,0.268 ± 0.152,0.282 ± 0.153,0.266 ± 0.143,NA
9,0.275 ± 0.156,0.276 ± 0.144,0.286 ± 0.156,0.279 ± 0.144,0.283 ± 0.149,0.274 ± 0.16,0.273 ± 0.154,0.288 ± 0.153,0.296 ± 0.158,NA


In [30]:
# group 1, ML + ST 

# read the table compiled in chapter 4. So this cell does not depend on that one to run, it just needs the table file.
df_new = read.table("tables/RF_swapped_set_all_methods_vs_reference.tsv", header=T)

# filter the above table to retain only rf values associated with trees coming from group 1 and method ME + SM 
df_group1_ML_ST = df_new  %>%
  filter(group == "group1_23th", method == "ST_ML")

# Group by the swap_species and swap_seq per swap combination and calculate mean rf_val and sd
df_group1_ML_ST_processed = df_group1_ML_ST %>%
    group_by(swap_species, swap_seq) %>%
    summarise(
        mean_rf = round(mean(rf_val), 3),
        sd_rf = round(sd(rf_val), 3),
        .groups = "drop"
    ) %>%
    # Combine mean and sd into one string
    mutate(mean_sd = paste0(mean_rf, " ± ", sd_rf)) %>%
    select(swap_species, swap_seq, mean_sd) %>%
    # Create a wide format 10x10 matrix/table
    pivot_wider(names_from = swap_seq, values_from = mean_sd, values_fill = NA) %>%
    # Arrange rows in order
    arrange(swap_species)

df_group1_ML_ST_processed
write.table(df_group1_ML_ST_processed, file = "tables/avg_RF_swapped_set_group1_ML_ST_vs_reference.tsv", quote = FALSE, row.names = FALSE, sep = "\t")

swap_species,2,3,4,5,6,7,8,9,10,15
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,0.281 ± 0.15,0.285 ± 0.149,0.284 ± 0.15,0.282 ± 0.15,0.28 ± 0.148,0.283 ± 0.153,0.288 ± 0.155,0.288 ± 0.153,0.288 ± 0.151,NA
2,0.284 ± 0.15,0.288 ± 0.15,0.283 ± 0.146,0.283 ± 0.152,0.287 ± 0.153,0.279 ± 0.147,0.288 ± 0.15,0.283 ± 0.151,0.28 ± 0.15,NA
3,0.287 ± 0.149,0.279 ± 0.151,0.281 ± 0.15,0.29 ± 0.16,0.292 ± 0.16,0.292 ± 0.148,0.285 ± 0.142,0.286 ± 0.154,0.282 ± 0.143,NA
4,0.286 ± 0.144,0.281 ± 0.158,0.288 ± 0.154,0.285 ± 0.151,0.279 ± 0.154,0.285 ± 0.148,0.278 ± 0.15,0.284 ± 0.154,0.282 ± 0.146,NA
5,0.284 ± 0.153,0.284 ± 0.152,0.291 ± 0.155,0.282 ± 0.153,0.273 ± 0.145,0.287 ± 0.15,0.284 ± 0.142,0.292 ± 0.159,0.287 ± 0.151,NA
6,0.285 ± 0.156,0.283 ± 0.148,0.292 ± 0.153,0.288 ± 0.152,0.293 ± 0.15,0.295 ± 0.15,0.286 ± 0.152,0.288 ± 0.145,0.286 ± 0.15,NA
7,0.287 ± 0.156,0.296 ± 0.157,0.286 ± 0.147,0.288 ± 0.157,0.284 ± 0.156,0.292 ± 0.165,0.288 ± 0.154,0.294 ± 0.156,0.291 ± 0.144,NA
8,0.278 ± 0.148,0.285 ± 0.143,0.287 ± 0.15,0.285 ± 0.148,0.29 ± 0.158,0.297 ± 0.154,0.283 ± 0.156,0.285 ± 0.149,0.285 ± 0.151,NA
9,0.285 ± 0.143,0.279 ± 0.153,0.291 ± 0.15,0.286 ± 0.141,0.288 ± 0.149,0.292 ± 0.166,0.287 ± 0.145,0.298 ± 0.153,0.292 ± 0.159,NA


In [31]:
# group 2, ME + SM 

# read the table compiled in chapter 4. So this cell does not depend on that one to run, it just needs the table file.
df_new = read.table("tables/RF_swapped_set_all_methods_vs_reference.tsv", header=T)

# filter the above table to retain only rf values associated with trees coming from group 1 and method ME + SM 
df_group2_ME_SM = df_new  %>%
  filter(group == "group2_22th", method == "SM_ME")

# Group by the swap_species and swap_seq per swap combination and calculate mean rf_val and sd
df_group2_ME_SM_processed = df_group2_ME_SM %>%
    group_by(swap_species, swap_seq) %>%
    summarise(
        mean_rf = round(mean(rf_val), 3),
        sd_rf = round(sd(rf_val), 3),
        .groups = "drop"
    ) %>%
    # Combine mean and sd into one string
    mutate(mean_sd = paste0(mean_rf, " ± ", sd_rf)) %>%
    select(swap_species, swap_seq, mean_sd) %>%
    # Create a wide format 10x10 matrix/table
    pivot_wider(names_from = swap_seq, values_from = mean_sd, values_fill = NA) %>%
    # Arrange rows in order
    arrange(swap_species)

df_group2_ME_SM_processed
write.table(df_group2_ME_SM_processed, file = "tables/avg_RF_swapped_set_group2_ME_SM_vs_reference.tsv", quote = FALSE, row.names = FALSE, sep = "\t")

swap_species,2,3,4,5,6,7,8,9,10,15
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,0.273 ± 0.151,0.276 ± 0.145,0.269 ± 0.147,0.273 ± 0.146,0.274 ± 0.141,0.272 ± 0.147,0.273 ± 0.152,0.278 ± 0.153,0.268 ± 0.146,NA
2,0.272 ± 0.154,0.27 ± 0.145,0.276 ± 0.154,0.272 ± 0.146,0.276 ± 0.158,0.274 ± 0.144,0.279 ± 0.149,0.27 ± 0.146,0.271 ± 0.142,NA
3,0.277 ± 0.145,0.273 ± 0.148,0.282 ± 0.146,0.278 ± 0.136,0.271 ± 0.147,0.275 ± 0.145,0.283 ± 0.15,0.263 ± 0.146,0.268 ± 0.142,NA
4,0.278 ± 0.148,0.277 ± 0.148,0.278 ± 0.144,0.267 ± 0.147,0.27 ± 0.148,0.28 ± 0.148,0.272 ± 0.139,0.273 ± 0.147,0.288 ± 0.139,NA
5,0.267 ± 0.149,0.271 ± 0.149,0.288 ± 0.142,0.278 ± 0.146,0.282 ± 0.154,0.276 ± 0.142,0.269 ± 0.154,0.265 ± 0.144,0.278 ± 0.151,NA
6,0.277 ± 0.144,0.275 ± 0.155,0.281 ± 0.138,0.278 ± 0.152,0.265 ± 0.147,0.278 ± 0.141,0.276 ± 0.158,0.288 ± 0.144,0.283 ± 0.143,NA
7,0.269 ± 0.143,0.274 ± 0.142,0.277 ± 0.156,0.275 ± 0.15,0.278 ± 0.152,0.276 ± 0.15,0.274 ± 0.138,0.269 ± 0.143,0.272 ± 0.144,NA
8,0.272 ± 0.133,0.268 ± 0.153,0.292 ± 0.143,0.291 ± 0.141,0.273 ± 0.156,0.279 ± 0.153,0.279 ± 0.15,0.279 ± 0.147,0.28 ± 0.149,NA
9,0.279 ± 0.148,0.275 ± 0.148,0.274 ± 0.143,0.28 ± 0.131,0.273 ± 0.144,0.302 ± 0.147,0.285 ± 0.153,0.292 ± 0.144,0.284 ± 0.145,NA


In [32]:
# group 2, ME + ST 

# read the table compiled in chapter 4. So this cell does not depend on that one to run, it just needs the table file.
df_new = read.table("tables/RF_swapped_set_all_methods_vs_reference.tsv", header=T)

# filter the above table to retain only rf values associated with trees coming from group 1 and method ME + ST 
df_group2_ME_ST = df_new  %>%
  filter(group == "group2_22th", method == "ST_ME")

# Group by the swap_species and swap_seq per swap combination and calculate mean rf_val and sd
df_group2_ME_ST_processed = df_group2_ME_ST %>%
    group_by(swap_species, swap_seq) %>%
    summarise(
        mean_rf = round(mean(rf_val), 3),
        sd_rf = round(sd(rf_val), 3),
        .groups = "drop"
    ) %>%
    # Combine mean and sd into one string
    mutate(mean_sd = paste0(mean_rf, " ± ", sd_rf)) %>%
    select(swap_species, swap_seq, mean_sd) %>%
    # Create a wide format 10x10 matrix/table
    pivot_wider(names_from = swap_seq, values_from = mean_sd, values_fill = NA) %>%
    # Arrange rows in order
    arrange(swap_species)


df_group2_ME_ST_processed
write.table(df_group2_ME_ST_processed, file = "tables/avg_RF_swapped_set_group2_ME_ST_vs_reference.tsv", quote = FALSE, row.names = FALSE, sep = "\t")

swap_species,2,3,4,5,6,7,8,9,10,15
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,0.273 ± 0.146,0.276 ± 0.148,0.272 ± 0.145,0.275 ± 0.15,0.279 ± 0.151,0.278 ± 0.149,0.275 ± 0.152,0.278 ± 0.145,0.276 ± 0.144,NA
2,0.275 ± 0.145,0.275 ± 0.147,0.279 ± 0.149,0.275 ± 0.142,0.273 ± 0.146,0.275 ± 0.15,0.281 ± 0.146,0.279 ± 0.151,0.276 ± 0.147,NA
3,0.272 ± 0.15,0.272 ± 0.142,0.278 ± 0.145,0.282 ± 0.143,0.275 ± 0.148,0.268 ± 0.147,0.273 ± 0.147,0.272 ± 0.145,0.275 ± 0.151,NA
4,0.275 ± 0.143,0.273 ± 0.155,0.276 ± 0.145,0.278 ± 0.146,0.278 ± 0.146,0.278 ± 0.146,0.282 ± 0.15,0.273 ± 0.148,0.283 ± 0.142,NA
5,0.281 ± 0.147,0.278 ± 0.145,0.278 ± 0.148,0.278 ± 0.149,0.283 ± 0.151,0.279 ± 0.147,0.28 ± 0.152,0.278 ± 0.147,0.284 ± 0.147,NA
6,0.278 ± 0.146,0.274 ± 0.151,0.281 ± 0.149,0.279 ± 0.155,0.282 ± 0.138,0.285 ± 0.144,0.281 ± 0.144,0.28 ± 0.145,0.278 ± 0.151,NA
7,0.277 ± 0.149,0.282 ± 0.152,0.275 ± 0.152,0.272 ± 0.148,0.273 ± 0.14,0.28 ± 0.147,0.286 ± 0.146,0.278 ± 0.142,0.276 ± 0.151,NA
8,0.28 ± 0.151,0.278 ± 0.148,0.283 ± 0.146,0.278 ± 0.152,0.283 ± 0.147,0.286 ± 0.141,0.286 ± 0.145,0.293 ± 0.154,0.287 ± 0.152,NA
9,0.279 ± 0.154,0.283 ± 0.149,0.277 ± 0.146,0.281 ± 0.145,0.283 ± 0.149,0.286 ± 0.152,0.284 ± 0.15,0.289 ± 0.155,0.279 ± 0.149,NA


In [33]:
# group 2, ML + SM 

# read the table compiled in chapter 4. So this cell does not depend on that one to run, it just needs the table file.
df_new = read.table("tables/RF_swapped_set_all_methods_vs_reference.tsv", header=T)

# filter the above table to retain only rf values associated with trees coming from group 1 and method ME + SM 
df_group2_ML_SM = df_new  %>%
  filter(group == "group2_22th", method == "SM_ML")

# Group by the swap_species and swap_seq per swap combination and calculate mean rf_val and sd
df_group2_ML_SM_processed = df_group2_ML_SM %>%
    group_by(swap_species, swap_seq) %>%
    summarise(
        mean_rf = round(mean(rf_val), 3),
        sd_rf = round(sd(rf_val), 3),
        .groups = "drop"
    ) %>%
    # Combine mean and sd into one string
    mutate(mean_sd = paste0(mean_rf, " ± ", sd_rf)) %>%
    select(swap_species, swap_seq, mean_sd) %>%
    # Create a wide format 10x10 matrix/table
    pivot_wider(names_from = swap_seq, values_from = mean_sd, values_fill = NA) %>%
    # Arrange rows in order
    arrange(swap_species)

df_group2_ML_SM_processed
write.table(df_group2_ML_SM_processed, file = "tables/avg_RF_swapped_set_group2_ML_SM_vs_reference.tsv", quote = FALSE, row.names = FALSE, sep = "\t")

swap_species,2,3,4,5,6,7,8,9,10,15
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,0.221 ± 0.145,0.221 ± 0.145,0.223 ± 0.141,0.218 ± 0.134,0.219 ± 0.144,0.218 ± 0.143,0.226 ± 0.141,0.218 ± 0.141,0.219 ± 0.141,NA
2,0.219 ± 0.144,0.223 ± 0.147,0.218 ± 0.143,0.224 ± 0.148,0.213 ± 0.15,0.218 ± 0.142,0.223 ± 0.145,0.223 ± 0.143,0.225 ± 0.15,NA
3,0.216 ± 0.142,0.217 ± 0.144,0.224 ± 0.143,0.214 ± 0.142,0.224 ± 0.138,0.222 ± 0.15,0.217 ± 0.149,0.218 ± 0.142,0.237 ± 0.146,NA
4,0.21 ± 0.142,0.228 ± 0.145,0.22 ± 0.142,0.218 ± 0.141,0.227 ± 0.144,0.225 ± 0.147,0.218 ± 0.15,0.217 ± 0.142,0.229 ± 0.143,NA
5,0.228 ± 0.145,0.223 ± 0.143,0.227 ± 0.141,0.219 ± 0.145,0.23 ± 0.149,0.221 ± 0.148,0.237 ± 0.141,0.228 ± 0.145,0.232 ± 0.139,NA
6,0.227 ± 0.141,0.226 ± 0.145,0.233 ± 0.139,0.228 ± 0.151,0.231 ± 0.143,0.229 ± 0.146,0.238 ± 0.144,0.238 ± 0.153,0.243 ± 0.159,NA
7,0.224 ± 0.138,0.224 ± 0.144,0.222 ± 0.149,0.222 ± 0.144,0.227 ± 0.147,0.24 ± 0.152,0.221 ± 0.136,0.23 ± 0.144,0.229 ± 0.145,NA
8,0.224 ± 0.141,0.224 ± 0.146,0.233 ± 0.144,0.227 ± 0.154,0.235 ± 0.137,0.249 ± 0.145,0.233 ± 0.153,0.24 ± 0.151,0.242 ± 0.148,NA
9,0.225 ± 0.153,0.243 ± 0.15,0.241 ± 0.151,0.245 ± 0.14,0.235 ± 0.155,0.242 ± 0.16,0.233 ± 0.144,0.243 ± 0.143,0.238 ± 0.149,NA


In [34]:
# group 2, ML + ST 

# read the table compiled in chapter 4. So this cell does not depend on that one to run, it just needs the table file.
df_new = read.table("tables/RF_swapped_set_all_methods_vs_reference.tsv", header=T)

# filter the above table to retain only rf values associated with trees coming from group 1 and method ME + SM 
df_group2_ML_ST = df_new  %>%
  filter(group == "group2_22th", method == "ST_ML")

# Group by the swap_species and swap_seq per swap combination and calculate mean rf_val and sd
df_group2_ML_ST_processed = df_group2_ML_ST %>%
    group_by(swap_species, swap_seq) %>%
    summarise(
        mean_rf = round(mean(rf_val), 3),
        sd_rf = round(sd(rf_val), 3),
        .groups = "drop"
    ) %>%
    # Combine mean and sd into one string
    mutate(mean_sd = paste0(mean_rf, " ± ", sd_rf)) %>%
    select(swap_species, swap_seq, mean_sd) %>%
    # Create a wide format 10x10 matrix/table
    pivot_wider(names_from = swap_seq, values_from = mean_sd, values_fill = NA) %>%
    # Arrange rows in order
    arrange(swap_species)

df_group2_ML_ST_processed
write.table(df_group2_ML_ST_processed, file = "tables/avg_RF_swapped_set_group2_ML_ST_vs_reference.tsv", quote = FALSE, row.names = FALSE, sep = "\t")

swap_species,2,3,4,5,6,7,8,9,10,15
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,0.236 ± 0.156,0.234 ± 0.159,0.234 ± 0.159,0.228 ± 0.152,0.238 ± 0.158,0.229 ± 0.15,0.237 ± 0.157,0.23 ± 0.154,0.236 ± 0.16,NA
2,0.234 ± 0.16,0.235 ± 0.156,0.238 ± 0.158,0.234 ± 0.157,0.231 ± 0.153,0.228 ± 0.149,0.233 ± 0.15,0.238 ± 0.153,0.231 ± 0.158,NA
3,0.234 ± 0.155,0.242 ± 0.161,0.231 ± 0.154,0.238 ± 0.152,0.233 ± 0.152,0.231 ± 0.152,0.235 ± 0.159,0.236 ± 0.152,0.238 ± 0.154,NA
4,0.233 ± 0.16,0.233 ± 0.154,0.234 ± 0.154,0.238 ± 0.155,0.237 ± 0.152,0.236 ± 0.158,0.233 ± 0.157,0.233 ± 0.149,0.237 ± 0.15,NA
5,0.239 ± 0.157,0.234 ± 0.154,0.233 ± 0.154,0.239 ± 0.152,0.238 ± 0.15,0.233 ± 0.154,0.246 ± 0.157,0.24 ± 0.152,0.245 ± 0.156,NA
6,0.238 ± 0.154,0.234 ± 0.152,0.236 ± 0.157,0.241 ± 0.147,0.233 ± 0.154,0.229 ± 0.141,0.241 ± 0.162,0.233 ± 0.145,0.238 ± 0.153,NA
7,0.232 ± 0.152,0.231 ± 0.144,0.238 ± 0.153,0.238 ± 0.158,0.241 ± 0.153,0.245 ± 0.159,0.23 ± 0.158,0.239 ± 0.144,0.236 ± 0.154,NA
8,0.241 ± 0.155,0.237 ± 0.157,0.233 ± 0.151,0.236 ± 0.146,0.242 ± 0.156,0.235 ± 0.145,0.238 ± 0.157,0.235 ± 0.156,0.243 ± 0.161,NA
9,0.245 ± 0.153,0.239 ± 0.154,0.238 ± 0.156,0.238 ± 0.141,0.233 ± 0.153,0.239 ± 0.158,0.242 ± 0.156,0.256 ± 0.155,0.233 ± 0.147,NA


In [35]:
# group 3, ME + SM 

# read the table compiled in chapter 4. So this cell does not depend on that one to run, it just needs the table file.
df_new = read.table("tables/RF_swapped_set_all_methods_vs_reference.tsv", header=T)

# filter the above table to retain only rf values associated with trees coming from group 1 and method ME + SM 
df_group3_ME_SM = df_new  %>%
  filter(group == "group3_20th", method == "SM_ME")

# Group by the swap_species and swap_seq per swap combination and calculate mean rf_val and sd
df_group3_ME_SM_processed = df_group3_ME_SM %>%
    group_by(swap_species, swap_seq) %>%
    summarise(
        mean_rf = round(mean(rf_val), 3),
        sd_rf = round(sd(rf_val), 3),
        .groups = "drop"
    ) %>%
    # Combine mean and sd into one string
    mutate(mean_sd = paste0(mean_rf, " ± ", sd_rf)) %>%
    select(swap_species, swap_seq, mean_sd) %>%
    # Create a wide format 10x10 matrix/table
    pivot_wider(names_from = swap_seq, values_from = mean_sd, values_fill = NA) %>%
    # Arrange rows in order
    arrange(swap_species)

df_group3_ME_SM_processed
write.table(df_group3_ME_SM_processed, file = "tables/avg_RF_swapped_set_group3_ME_SM_vs_reference.tsv", quote = FALSE, row.names = FALSE, sep = "\t")

swap_species,2,3,4,5,6,7,8,9,10,15
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,0.228 ± 0.124,0.231 ± 0.129,0.233 ± 0.117,0.232 ± 0.125,0.236 ± 0.125,0.225 ± 0.125,0.234 ± 0.128,0.229 ± 0.128,0.22 ± 0.129,NA
2,0.235 ± 0.124,0.24 ± 0.133,0.239 ± 0.131,0.23 ± 0.121,0.246 ± 0.13,0.232 ± 0.131,0.233 ± 0.132,0.226 ± 0.128,0.236 ± 0.125,NA
3,0.228 ± 0.125,0.237 ± 0.12,0.23 ± 0.129,0.231 ± 0.131,0.239 ± 0.134,0.235 ± 0.131,0.222 ± 0.129,0.232 ± 0.127,0.229 ± 0.129,NA
4,0.232 ± 0.124,0.232 ± 0.122,0.238 ± 0.127,0.238 ± 0.136,0.238 ± 0.135,0.236 ± 0.129,0.228 ± 0.131,0.222 ± 0.138,0.231 ± 0.119,NA
5,0.23 ± 0.119,0.224 ± 0.132,0.226 ± 0.132,0.234 ± 0.131,0.233 ± 0.132,0.229 ± 0.135,0.229 ± 0.133,0.222 ± 0.128,0.218 ± 0.121,NA
6,0.221 ± 0.116,0.222 ± 0.137,0.23 ± 0.135,0.226 ± 0.129,0.228 ± 0.135,0.241 ± 0.141,0.252 ± 0.13,0.224 ± 0.13,0.243 ± 0.138,NA
7,0.234 ± 0.131,0.229 ± 0.132,0.235 ± 0.127,0.233 ± 0.121,0.248 ± 0.142,0.241 ± 0.133,0.235 ± 0.132,0.233 ± 0.13,0.231 ± 0.132,NA
8,0.233 ± 0.132,0.243 ± 0.127,0.228 ± 0.128,0.244 ± 0.139,0.224 ± 0.134,0.23 ± 0.134,0.241 ± 0.137,0.243 ± 0.131,0.249 ± 0.137,NA
9,0.235 ± 0.138,0.235 ± 0.124,0.233 ± 0.135,0.239 ± 0.124,0.246 ± 0.128,0.241 ± 0.152,0.247 ± 0.125,0.222 ± 0.137,0.233 ± 0.133,NA


In [36]:
# group 3, ME + ST 

# read the table compiled in chapter 4. So this cell does not depend on that one to run, it just needs the table file.
df_new = read.table("tables/RF_swapped_set_all_methods_vs_reference.tsv", header=T)

# filter the above table to retain only rf values associated with trees coming from group 1 and method ME + ST 
df_group3_ME_ST = df_new  %>%
  filter(group == "group3_20th", method == "ST_ME")

# Group by the swap_species and swap_seq per swap combination and calculate mean rf_val and sd
df_group3_ME_ST_processed = df_group3_ME_ST %>%
    group_by(swap_species, swap_seq) %>%
    summarise(
        mean_rf = round(mean(rf_val), 3),
        sd_rf = round(sd(rf_val), 3),
        .groups = "drop"
    ) %>%
    # Combine mean and sd into one string
    mutate(mean_sd = paste0(mean_rf, " ± ", sd_rf)) %>%
    select(swap_species, swap_seq, mean_sd) %>%
    # Create a wide format 10x10 matrix/table
    pivot_wider(names_from = swap_seq, values_from = mean_sd, values_fill = NA) %>%
    # Arrange rows in order
    arrange(swap_species)


df_group3_ME_ST_processed
write.table(df_group3_ME_ST_processed, file = "tables/avg_RF_swapped_set_group3_ME_ST_vs_reference.tsv", quote = FALSE, row.names = FALSE, sep = "\t")

swap_species,2,3,4,5,6,7,8,9,10,15
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,0.239 ± 0.143,0.236 ± 0.143,0.236 ± 0.145,0.238 ± 0.146,0.238 ± 0.146,0.238 ± 0.141,0.238 ± 0.137,0.244 ± 0.142,0.239 ± 0.14,NA
2,0.238 ± 0.143,0.236 ± 0.146,0.229 ± 0.141,0.243 ± 0.143,0.247 ± 0.149,0.238 ± 0.142,0.246 ± 0.146,0.237 ± 0.143,0.236 ± 0.138,NA
3,0.238 ± 0.143,0.236 ± 0.139,0.238 ± 0.147,0.243 ± 0.142,0.239 ± 0.141,0.243 ± 0.142,0.239 ± 0.15,0.239 ± 0.148,0.24 ± 0.137,NA
4,0.238 ± 0.148,0.238 ± 0.142,0.239 ± 0.137,0.247 ± 0.143,0.233 ± 0.145,0.251 ± 0.142,0.235 ± 0.138,0.243 ± 0.149,0.238 ± 0.139,NA
5,0.238 ± 0.14,0.227 ± 0.139,0.238 ± 0.14,0.243 ± 0.144,0.237 ± 0.148,0.244 ± 0.143,0.24 ± 0.141,0.235 ± 0.144,0.242 ± 0.138,NA
6,0.231 ± 0.144,0.229 ± 0.149,0.244 ± 0.143,0.244 ± 0.142,0.235 ± 0.147,0.245 ± 0.145,0.24 ± 0.146,0.24 ± 0.147,0.25 ± 0.14,NA
7,0.238 ± 0.143,0.244 ± 0.142,0.243 ± 0.14,0.237 ± 0.148,0.234 ± 0.14,0.248 ± 0.149,0.248 ± 0.136,0.243 ± 0.145,0.24 ± 0.143,NA
8,0.241 ± 0.138,0.247 ± 0.132,0.251 ± 0.139,0.243 ± 0.14,0.243 ± 0.144,0.245 ± 0.141,0.257 ± 0.146,0.252 ± 0.139,0.253 ± 0.136,NA
9,0.238 ± 0.139,0.25 ± 0.141,0.245 ± 0.132,0.251 ± 0.139,0.259 ± 0.148,0.248 ± 0.135,0.25 ± 0.136,0.252 ± 0.139,0.243 ± 0.144,NA


In [37]:
# group 3, ML + SM 

# read the table compiled in chapter 4. So this cell does not depend on that one to run, it just needs the table file.
df_new = read.table("tables/RF_swapped_set_all_methods_vs_reference.tsv", header=T)

# filter the above table to retain only rf values associated with trees coming from group 1 and method ME + SM 
df_group3_ML_SM = df_new  %>%
  filter(group == "group3_20th", method == "SM_ML")

# Group by the swap_species and swap_seq per swap combination and calculate mean rf_val and sd
df_group3_ML_SM_processed = df_group3_ML_SM %>%
    group_by(swap_species, swap_seq) %>%
    summarise(
        mean_rf = round(mean(rf_val), 3),
        sd_rf = round(sd(rf_val), 3),
        .groups = "drop"
    ) %>%
    # Combine mean and sd into one string
    mutate(mean_sd = paste0(mean_rf, " ± ", sd_rf)) %>%
    select(swap_species, swap_seq, mean_sd) %>%
    # Create a wide format 10x10 matrix/table
    pivot_wider(names_from = swap_seq, values_from = mean_sd, values_fill = NA) %>%
    # Arrange rows in order
    arrange(swap_species)

df_group3_ML_SM_processed
write.table(df_group3_ML_SM_processed, file = "tables/avg_RF_swapped_set_group3_ML_SM_vs_reference.tsv", quote = FALSE, row.names = FALSE, sep = "\t")

swap_species,2,3,4,5,6,7,8,9,10,15
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,0.2 ± 0.136,0.203 ± 0.137,0.205 ± 0.147,0.2 ± 0.144,0.205 ± 0.137,0.2 ± 0.134,0.203 ± 0.14,0.198 ± 0.132,0.203 ± 0.142,NA
2,0.198 ± 0.14,0.201 ± 0.144,0.205 ± 0.139,0.203 ± 0.146,0.205 ± 0.143,0.208 ± 0.136,0.195 ± 0.142,0.209 ± 0.142,0.201 ± 0.14,NA
3,0.202 ± 0.149,0.202 ± 0.137,0.196 ± 0.138,0.202 ± 0.138,0.202 ± 0.127,0.196 ± 0.137,0.206 ± 0.138,0.201 ± 0.148,0.213 ± 0.139,NA
4,0.206 ± 0.134,0.201 ± 0.131,0.206 ± 0.138,0.197 ± 0.135,0.208 ± 0.132,0.212 ± 0.145,0.21 ± 0.141,0.202 ± 0.134,0.214 ± 0.145,NA
5,0.202 ± 0.146,0.208 ± 0.138,0.21 ± 0.141,0.208 ± 0.14,0.198 ± 0.139,0.199 ± 0.138,0.206 ± 0.142,0.195 ± 0.128,0.201 ± 0.145,NA
6,0.213 ± 0.138,0.204 ± 0.13,0.21 ± 0.143,0.202 ± 0.134,0.209 ± 0.153,0.208 ± 0.146,0.205 ± 0.129,0.209 ± 0.147,0.209 ± 0.147,NA
7,0.206 ± 0.141,0.21 ± 0.143,0.198 ± 0.136,0.213 ± 0.148,0.203 ± 0.133,0.211 ± 0.136,0.222 ± 0.143,0.205 ± 0.12,0.213 ± 0.133,NA
8,0.203 ± 0.143,0.206 ± 0.141,0.208 ± 0.132,0.211 ± 0.132,0.214 ± 0.148,0.213 ± 0.142,0.24 ± 0.159,0.218 ± 0.146,0.221 ± 0.16,NA
9,0.2 ± 0.131,0.203 ± 0.133,0.204 ± 0.137,0.223 ± 0.151,0.214 ± 0.141,0.208 ± 0.148,0.228 ± 0.155,0.224 ± 0.154,0.228 ± 0.155,NA


In [38]:
# group 3, ML + ST 

# read the table compiled in chapter 4. So this cell does not depend on that one to run, it just needs the table file.
df_new = read.table("tables/RF_swapped_set_all_methods_vs_reference.tsv", header=T)

# filter the above table to retain only rf values associated with trees coming from group 1 and method ME + SM 
df_group3_ML_ST = df_new  %>%
  filter(group == "group3_20th", method == "ST_ML")

# Group by the swap_species and swap_seq per swap combination and calculate mean rf_val and sd
df_group3_ML_ST_processed = df_group3_ML_ST %>%
    group_by(swap_species, swap_seq) %>%
    summarise(
        mean_rf = round(mean(rf_val), 3),
        sd_rf = round(sd(rf_val), 3),
        .groups = "drop"
    ) %>%
    # Combine mean and sd into one string
    mutate(mean_sd = paste0(mean_rf, " ± ", sd_rf)) %>%
    select(swap_species, swap_seq, mean_sd) %>%
    # Create a wide format 10x10 matrix/table
    pivot_wider(names_from = swap_seq, values_from = mean_sd, values_fill = NA) %>%
    # Arrange rows in order
    arrange(swap_species)

df_group3_ML_ST_processed
write.table(df_group3_ML_ST_processed, file = "tables/avg_RF_swapped_set_group3_ML_ST_vs_reference.tsv", quote = FALSE, row.names = FALSE, sep = "\t")

swap_species,2,3,4,5,6,7,8,9,10,15
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,0.213 ± 0.134,0.208 ± 0.137,0.211 ± 0.137,0.212 ± 0.131,0.213 ± 0.13,0.212 ± 0.136,0.209 ± 0.132,0.208 ± 0.136,0.212 ± 0.135,NA
2,0.208 ± 0.135,0.213 ± 0.136,0.213 ± 0.139,0.212 ± 0.132,0.213 ± 0.139,0.21 ± 0.134,0.21 ± 0.136,0.209 ± 0.133,0.209 ± 0.124,NA
3,0.208 ± 0.137,0.214 ± 0.131,0.208 ± 0.13,0.21 ± 0.135,0.207 ± 0.13,0.208 ± 0.134,0.216 ± 0.133,0.213 ± 0.132,0.21 ± 0.137,NA
4,0.213 ± 0.136,0.21 ± 0.137,0.21 ± 0.128,0.213 ± 0.122,0.208 ± 0.136,0.208 ± 0.122,0.209 ± 0.134,0.205 ± 0.132,0.207 ± 0.125,NA
5,0.209 ± 0.134,0.207 ± 0.131,0.21 ± 0.136,0.215 ± 0.127,0.206 ± 0.133,0.2 ± 0.135,0.207 ± 0.137,0.208 ± 0.126,0.207 ± 0.124,NA
6,0.208 ± 0.135,0.209 ± 0.132,0.214 ± 0.131,0.208 ± 0.138,0.216 ± 0.136,0.209 ± 0.13,0.218 ± 0.139,0.209 ± 0.134,0.214 ± 0.137,NA
7,0.212 ± 0.141,0.218 ± 0.137,0.203 ± 0.134,0.205 ± 0.136,0.208 ± 0.129,0.215 ± 0.131,0.217 ± 0.136,0.206 ± 0.125,0.213 ± 0.143,NA
8,0.21 ± 0.134,0.218 ± 0.138,0.212 ± 0.135,0.208 ± 0.127,0.226 ± 0.141,0.207 ± 0.128,0.223 ± 0.144,0.208 ± 0.132,0.21 ± 0.143,NA
9,0.211 ± 0.135,0.206 ± 0.127,0.204 ± 0.133,0.213 ± 0.129,0.213 ± 0.129,0.212 ± 0.152,0.209 ± 0.137,0.219 ± 0.138,0.212 ± 0.133,NA


In [39]:
# group 4, ME + SM 

# read the table compiled in chapter 4. So this cell does not depend on that one to run, it just needs the table file.
df_new = read.table("tables/RF_swapped_set_all_methods_vs_reference.tsv", header=T)

# filter the above table to retain only rf values associated with trees coming from group 1 and method ME + SM 
df_group4_ME_SM = df_new  %>%
  filter(group == "group4_17th", method == "SM_ME")

# Group by the swap_species and swap_seq per swap combination and calculate mean rf_val and sd
df_group4_ME_SM_processed = df_group4_ME_SM %>%
    group_by(swap_species, swap_seq) %>%
    summarise(
        mean_rf = round(mean(rf_val), 3),
        sd_rf = round(sd(rf_val), 3),
        .groups = "drop"
    ) %>%
    # Combine mean and sd into one string
    mutate(mean_sd = paste0(mean_rf, " ± ", sd_rf)) %>%
    select(swap_species, swap_seq, mean_sd) %>%
    # Create a wide format 10x10 matrix/table
    pivot_wider(names_from = swap_seq, values_from = mean_sd, values_fill = NA) %>%
    # Arrange rows in order
    arrange(swap_species)

df_group4_ME_SM_processed
write.table(df_group4_ME_SM_processed, file = "tables/avg_RF_swapped_set_group4_ME_SM_vs_reference.tsv", quote = FALSE, row.names = FALSE, sep = "\t")

swap_species,2,3,4,5,6,7,8,9,10,15
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,0.206 ± 0.137,0.203 ± 0.136,0.205 ± 0.135,0.2 ± 0.138,0.203 ± 0.133,0.205 ± 0.138,0.21 ± 0.136,0.203 ± 0.136,0.203 ± 0.126,NA
2,0.2 ± 0.13,0.206 ± 0.134,0.2 ± 0.14,0.208 ± 0.134,0.2 ± 0.132,0.198 ± 0.126,0.206 ± 0.134,0.203 ± 0.137,0.209 ± 0.137,NA
3,0.205 ± 0.131,0.206 ± 0.14,0.208 ± 0.137,0.21 ± 0.135,0.204 ± 0.141,0.193 ± 0.14,0.206 ± 0.139,0.199 ± 0.134,0.208 ± 0.134,NA
4,0.208 ± 0.14,0.203 ± 0.138,0.208 ± 0.135,0.198 ± 0.138,0.204 ± 0.131,0.203 ± 0.135,0.203 ± 0.137,0.205 ± 0.131,0.207 ± 0.139,NA
5,0.2 ± 0.137,0.205 ± 0.136,0.201 ± 0.134,0.203 ± 0.14,0.2 ± 0.137,0.2 ± 0.13,0.208 ± 0.134,0.199 ± 0.131,0.195 ± 0.137,NA
6,0.213 ± 0.133,0.205 ± 0.131,0.205 ± 0.13,0.203 ± 0.142,0.19 ± 0.14,0.191 ± 0.144,0.201 ± 0.14,0.203 ± 0.141,0.207 ± 0.123,NA
7,0.207 ± 0.137,0.202 ± 0.136,0.205 ± 0.15,0.202 ± 0.135,0.193 ± 0.138,0.198 ± 0.134,0.203 ± 0.134,0.199 ± 0.14,0.196 ± 0.132,NA
8,0.206 ± 0.14,0.21 ± 0.141,0.201 ± 0.133,0.21 ± 0.142,0.201 ± 0.143,0.207 ± 0.135,0.204 ± 0.143,0.193 ± 0.141,0.184 ± 0.126,NA
9,0.199 ± 0.141,0.207 ± 0.133,0.206 ± 0.14,0.204 ± 0.136,0.201 ± 0.135,0.199 ± 0.132,0.208 ± 0.129,0.202 ± 0.137,0.207 ± 0.145,NA


In [40]:
# group 4, ME + ST 

# read the table compiled in chapter 4. So this cell does not depend on that one to run, it just needs the table file.
df_new = read.table("tables/RF_swapped_set_all_methods_vs_reference.tsv", header=T)

# filter the above table to retain only rf values associated with trees coming from group 1 and method ME + ST 
df_group4_ME_ST = df_new  %>%
  filter(group == "group4_17th", method == "ST_ME")

# Group by the swap_species and swap_seq per swap combination and calculate mean rf_val and sd
df_group4_ME_ST_processed = df_group4_ME_ST %>%
    group_by(swap_species, swap_seq) %>%
    summarise(
        mean_rf = round(mean(rf_val), 3),
        sd_rf = round(sd(rf_val), 3),
        .groups = "drop"
    ) %>%
    # Combine mean and sd into one string
    mutate(mean_sd = paste0(mean_rf, " ± ", sd_rf)) %>%
    select(swap_species, swap_seq, mean_sd) %>%
    # Create a wide format 10x10 matrix/table
    pivot_wider(names_from = swap_seq, values_from = mean_sd, values_fill = NA) %>%
    # Arrange rows in order
    arrange(swap_species)


df_group4_ME_ST_processed
write.table(df_group4_ME_ST_processed, file = "tables/avg_RF_swapped_set_group4_ME_ST_vs_reference.tsv", quote = FALSE, row.names = FALSE, sep = "\t")

swap_species,2,3,4,5,6,7,8,9,10,15
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,0.204 ± 0.142,0.205 ± 0.138,0.207 ± 0.138,0.2 ± 0.142,0.2 ± 0.143,0.202 ± 0.141,0.204 ± 0.142,0.2 ± 0.145,0.203 ± 0.143,NA
2,0.204 ± 0.142,0.204 ± 0.139,0.205 ± 0.14,0.206 ± 0.145,0.207 ± 0.141,0.208 ± 0.142,0.205 ± 0.14,0.202 ± 0.143,0.205 ± 0.143,NA
3,0.203 ± 0.145,0.204 ± 0.143,0.206 ± 0.143,0.198 ± 0.136,0.204 ± 0.141,0.195 ± 0.134,0.204 ± 0.141,0.202 ± 0.141,0.21 ± 0.143,NA
4,0.203 ± 0.145,0.202 ± 0.141,0.206 ± 0.145,0.203 ± 0.14,0.208 ± 0.141,0.203 ± 0.142,0.205 ± 0.133,0.202 ± 0.139,0.208 ± 0.143,NA
5,0.2 ± 0.143,0.204 ± 0.138,0.207 ± 0.142,0.205 ± 0.14,0.204 ± 0.141,0.202 ± 0.138,0.203 ± 0.138,0.202 ± 0.138,0.203 ± 0.131,NA
6,0.205 ± 0.14,0.203 ± 0.138,0.203 ± 0.138,0.208 ± 0.144,0.201 ± 0.143,0.203 ± 0.143,0.203 ± 0.141,0.205 ± 0.139,0.203 ± 0.131,NA
7,0.201 ± 0.134,0.201 ± 0.138,0.206 ± 0.139,0.201 ± 0.145,0.208 ± 0.142,0.206 ± 0.143,0.206 ± 0.14,0.206 ± 0.137,0.196 ± 0.145,NA
8,0.2 ± 0.14,0.203 ± 0.141,0.203 ± 0.146,0.202 ± 0.14,0.213 ± 0.142,0.203 ± 0.138,0.213 ± 0.142,0.206 ± 0.137,0.206 ± 0.142,NA
9,0.202 ± 0.137,0.203 ± 0.137,0.203 ± 0.14,0.203 ± 0.145,0.203 ± 0.134,0.2 ± 0.137,0.196 ± 0.133,0.207 ± 0.137,0.22 ± 0.139,NA


In [41]:
# group 4, ML + SM 

# read the table compiled in chapter 4. So this cell does not depend on that one to run, it just needs the table file.
df_new = read.table("tables/RF_swapped_set_all_methods_vs_reference.tsv", header=T)

# filter the above table to retain only rf values associated with trees coming from group 1 and method ME + SM 
df_group4_ML_SM = df_new  %>%
  filter(group == "group4_17th", method == "SM_ML")

# Group by the swap_species and swap_seq per swap combination and calculate mean rf_val and sd
df_group4_ML_SM_processed = df_group4_ML_SM %>%
    group_by(swap_species, swap_seq) %>%
    summarise(
        mean_rf = round(mean(rf_val), 3),
        sd_rf = round(sd(rf_val), 3),
        .groups = "drop"
    ) %>%
    # Combine mean and sd into one string
    mutate(mean_sd = paste0(mean_rf, " ± ", sd_rf)) %>%
    select(swap_species, swap_seq, mean_sd) %>%
    # Create a wide format 10x10 matrix/table
    pivot_wider(names_from = swap_seq, values_from = mean_sd, values_fill = NA) %>%
    # Arrange rows in order
    arrange(swap_species)

df_group4_ML_SM_processed
write.table(df_group4_ML_SM_processed, file = "tables/avg_RF_swapped_set_group4_ML_SM_vs_reference.tsv", quote = FALSE, row.names = FALSE, sep = "\t")

swap_species,2,3,4,5,6,7,8,9,10,15
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,0.148 ± 0.123,0.151 ± 0.122,0.156 ± 0.123,0.151 ± 0.126,0.15 ± 0.119,0.153 ± 0.122,0.151 ± 0.122,0.144 ± 0.118,0.154 ± 0.118,NA
2,0.151 ± 0.122,0.148 ± 0.119,0.151 ± 0.122,0.151 ± 0.12,0.162 ± 0.129,0.156 ± 0.122,0.151 ± 0.122,0.153 ± 0.117,0.154 ± 0.123,NA
3,0.154 ± 0.122,0.153 ± 0.121,0.153 ± 0.127,0.156 ± 0.122,0.157 ± 0.119,0.149 ± 0.119,0.162 ± 0.131,0.158 ± 0.119,0.148 ± 0.115,NA
4,0.15 ± 0.122,0.152 ± 0.118,0.149 ± 0.119,0.156 ± 0.127,0.155 ± 0.12,0.155 ± 0.12,0.156 ± 0.118,0.145 ± 0.126,0.157 ± 0.122,NA
5,0.156 ± 0.125,0.151 ± 0.124,0.16 ± 0.118,0.158 ± 0.125,0.152 ± 0.124,0.147 ± 0.117,0.163 ± 0.12,0.153 ± 0.116,0.147 ± 0.118,NA
6,0.164 ± 0.127,0.165 ± 0.124,0.152 ± 0.117,0.158 ± 0.118,0.159 ± 0.127,0.162 ± 0.131,0.172 ± 0.132,0.153 ± 0.115,0.16 ± 0.12,NA
7,0.156 ± 0.123,0.157 ± 0.125,0.163 ± 0.121,0.167 ± 0.12,0.168 ± 0.127,0.152 ± 0.12,0.158 ± 0.122,0.152 ± 0.12,0.152 ± 0.113,NA
8,0.15 ± 0.118,0.159 ± 0.116,0.157 ± 0.116,0.16 ± 0.137,0.163 ± 0.121,0.178 ± 0.123,0.164 ± 0.118,0.168 ± 0.128,0.179 ± 0.123,NA
9,0.151 ± 0.12,0.155 ± 0.12,0.163 ± 0.126,0.164 ± 0.123,0.162 ± 0.128,0.178 ± 0.128,0.174 ± 0.114,0.166 ± 0.124,0.173 ± 0.121,NA


In [42]:
# group 4, ML + ST 

# read the table compiled in chapter 4. So this cell does not depend on that one to run, it just needs the table file.
df_new = read.table("tables/RF_swapped_set_all_methods_vs_reference.tsv", header=T)

# filter the above table to retain only rf values associated with trees coming from group 1 and method ME + SM 
df_group4_ML_ST = df_new  %>%
  filter(group == "group4_17th", method == "ST_ML")

# Group by the swap_species and swap_seq per swap combination and calculate mean rf_val and sd
df_group4_ML_ST_processed = df_group4_ML_ST %>%
    group_by(swap_species, swap_seq) %>%
    summarise(
        mean_rf = round(mean(rf_val), 3),
        sd_rf = round(sd(rf_val), 3),
        .groups = "drop"
    ) %>%
    # Combine mean and sd into one string
    mutate(mean_sd = paste0(mean_rf, " ± ", sd_rf)) %>%
    select(swap_species, swap_seq, mean_sd) %>%
    # Create a wide format 10x10 matrix/table
    pivot_wider(names_from = swap_seq, values_from = mean_sd, values_fill = NA) %>%
    # Arrange rows in order
    arrange(swap_species)

df_group4_ML_ST_processed
write.table(df_group4_ML_ST_processed, file = "tables/avg_RF_swapped_set_group4_ML_ST_vs_reference.tsv", quote = FALSE, row.names = FALSE, sep = "\t")

swap_species,2,3,4,5,6,7,8,9,10,15
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,0.165 ± 0.127,0.166 ± 0.128,0.167 ± 0.128,0.165 ± 0.129,0.163 ± 0.126,0.165 ± 0.126,0.169 ± 0.127,0.167 ± 0.128,0.163 ± 0.124,NA
2,0.162 ± 0.128,0.162 ± 0.13,0.163 ± 0.127,0.164 ± 0.129,0.164 ± 0.129,0.175 ± 0.133,0.169 ± 0.124,0.17 ± 0.129,0.166 ± 0.132,NA
3,0.164 ± 0.128,0.162 ± 0.126,0.168 ± 0.13,0.17 ± 0.124,0.168 ± 0.132,0.168 ± 0.128,0.167 ± 0.131,0.16 ± 0.123,0.168 ± 0.129,NA
4,0.173 ± 0.127,0.173 ± 0.13,0.166 ± 0.128,0.169 ± 0.135,0.164 ± 0.124,0.171 ± 0.133,0.169 ± 0.128,0.163 ± 0.131,0.165 ± 0.127,NA
5,0.165 ± 0.129,0.164 ± 0.128,0.164 ± 0.126,0.167 ± 0.129,0.171 ± 0.129,0.175 ± 0.129,0.171 ± 0.128,0.168 ± 0.13,0.165 ± 0.129,NA
6,0.173 ± 0.128,0.163 ± 0.124,0.166 ± 0.131,0.173 ± 0.128,0.167 ± 0.126,0.173 ± 0.13,0.17 ± 0.124,0.168 ± 0.133,0.175 ± 0.135,NA
7,0.167 ± 0.128,0.167 ± 0.126,0.172 ± 0.127,0.178 ± 0.129,0.168 ± 0.123,0.169 ± 0.128,0.164 ± 0.132,0.174 ± 0.13,0.168 ± 0.131,NA
8,0.16 ± 0.122,0.163 ± 0.128,0.169 ± 0.127,0.169 ± 0.125,0.172 ± 0.128,0.171 ± 0.136,0.169 ± 0.132,0.169 ± 0.131,0.178 ± 0.136,NA
9,0.163 ± 0.124,0.171 ± 0.121,0.168 ± 0.125,0.169 ± 0.129,0.169 ± 0.123,0.168 ± 0.121,0.171 ± 0.123,0.173 ± 0.129,0.168 ± 0.125,NA


In [43]:
# group 5, ME + SM 

# read the table compiled in chapter 4. So this cell does not depend on that one to run, it just needs the table file.
df_new = read.table("tables/RF_swapped_set_all_methods_vs_reference.tsv", header=T)

# filter the above table to retain only rf values associated with trees coming from group 1 and method ME + SM 
df_group5_ME_SM = df_new  %>%
  filter(group == "group5_11th", method == "SM_ME")

# Group by the swap_species and swap_seq per swap combination and calculate mean rf_val and sd
df_group5_ME_SM_processed = df_group5_ME_SM %>%
    group_by(swap_species, swap_seq) %>%
    summarise(
        mean_rf = round(mean(rf_val), 3),
        sd_rf = round(sd(rf_val), 3),
        .groups = "drop"
    ) %>%
    # Combine mean and sd into one string
    mutate(mean_sd = paste0(mean_rf, " ± ", sd_rf)) %>%
    select(swap_species, swap_seq, mean_sd) %>%
    # Create a wide format 10x10 matrix/table
    pivot_wider(names_from = swap_seq, values_from = mean_sd, values_fill = NA) %>%
    # Arrange rows in order
    arrange(swap_species)

df_group5_ME_SM_processed
write.table(df_group5_ME_SM_processed, file = "tables/avg_RF_swapped_set_group5_ME_SM_vs_reference.tsv", quote = FALSE, row.names = FALSE, sep = "\t")

swap_species,2,3,4,5,6,7,8,9,10,15
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,0.143 ± 0.103,0.144 ± 0.102,0.155 ± 0.105,0.148 ± 0.1,0.15 ± 0.108,0.143 ± 0.105,0.148 ± 0.112,0.153 ± 0.106,0.141 ± 0.104,NA
2,0.148 ± 0.104,0.147 ± 0.104,0.153 ± 0.105,0.141 ± 0.099,0.154 ± 0.099,0.154 ± 0.11,0.149 ± 0.11,0.152 ± 0.103,0.148 ± 0.105,NA
3,0.147 ± 0.109,0.153 ± 0.106,0.16 ± 0.112,0.147 ± 0.103,0.152 ± 0.105,0.143 ± 0.1,0.142 ± 0.102,0.146 ± 0.112,0.153 ± 0.107,NA
4,0.154 ± 0.111,0.148 ± 0.105,0.141 ± 0.1,0.138 ± 0.103,0.15 ± 0.109,0.144 ± 0.104,0.143 ± 0.109,0.15 ± 0.103,0.155 ± 0.112,NA
5,0.146 ± 0.105,0.151 ± 0.106,0.152 ± 0.096,0.147 ± 0.103,0.146 ± 0.1,0.152 ± 0.111,0.148 ± 0.105,0.145 ± 0.101,0.156 ± 0.094,NA
6,0.158 ± 0.113,0.136 ± 0.095,0.156 ± 0.108,0.147 ± 0.105,0.147 ± 0.105,0.147 ± 0.104,0.148 ± 0.1,0.175 ± 0.108,0.152 ± 0.108,NA
7,0.143 ± 0.105,0.161 ± 0.107,0.151 ± 0.1,0.151 ± 0.101,0.155 ± 0.109,0.148 ± 0.106,0.168 ± 0.111,0.148 ± 0.102,0.154 ± 0.104,NA
8,0.152 ± 0.102,0.148 ± 0.108,0.158 ± 0.104,0.15 ± 0.108,0.148 ± 0.106,0.147 ± 0.107,0.159 ± 0.121,0.15 ± 0.11,0.155 ± 0.116,NA
9,0.152 ± 0.099,0.157 ± 0.104,0.153 ± 0.104,0.161 ± 0.11,0.154 ± 0.107,0.158 ± 0.109,0.16 ± 0.111,0.156 ± 0.103,0.156 ± 0.112,NA


In [44]:
# group 5, ME + ST 

# read the table compiled in chapter 4. So this cell does not depend on that one to run, it just needs the table file.
df_new = read.table("tables/RF_swapped_set_all_methods_vs_reference.tsv", header=T)

# filter the above table to retain only rf values associated with trees coming from group 1 and method ME + ST 
df_group5_ME_ST = df_new  %>%
  filter(group == "group5_11th", method == "ST_ME")

# Group by the swap_species and swap_seq per swap combination and calculate mean rf_val and sd
df_group5_ME_ST_processed = df_group5_ME_ST %>%
    group_by(swap_species, swap_seq) %>%
    summarise(
        mean_rf = round(mean(rf_val), 3),
        sd_rf = round(sd(rf_val), 3),
        .groups = "drop"
    ) %>%
    # Combine mean and sd into one string
    mutate(mean_sd = paste0(mean_rf, " ± ", sd_rf)) %>%
    select(swap_species, swap_seq, mean_sd) %>%
    # Create a wide format 10x10 matrix/table
    pivot_wider(names_from = swap_seq, values_from = mean_sd, values_fill = NA) %>%
    # Arrange rows in order
    arrange(swap_species)


df_group5_ME_ST_processed
write.table(df_group5_ME_ST_processed, file = "tables/avg_RF_swapped_set_group5_ME_ST_vs_reference.tsv", quote = FALSE, row.names = FALSE, sep = "\t")

swap_species,2,3,4,5,6,7,8,9,10,15
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,0.154 ± 0.107,0.151 ± 0.105,0.153 ± 0.104,0.153 ± 0.104,0.155 ± 0.103,0.148 ± 0.101,0.149 ± 0.103,0.155 ± 0.108,0.155 ± 0.108,NA
2,0.153 ± 0.108,0.154 ± 0.105,0.16 ± 0.108,0.155 ± 0.109,0.152 ± 0.108,0.156 ± 0.108,0.153 ± 0.1,0.157 ± 0.103,0.155 ± 0.106,NA
3,0.155 ± 0.103,0.153 ± 0.111,0.149 ± 0.104,0.155 ± 0.105,0.161 ± 0.107,0.154 ± 0.099,0.148 ± 0.105,0.154 ± 0.105,0.153 ± 0.108,NA
4,0.158 ± 0.11,0.154 ± 0.106,0.151 ± 0.101,0.153 ± 0.103,0.152 ± 0.104,0.148 ± 0.108,0.15 ± 0.102,0.156 ± 0.103,0.15 ± 0.105,NA
5,0.15 ± 0.104,0.156 ± 0.104,0.154 ± 0.105,0.157 ± 0.101,0.153 ± 0.107,0.154 ± 0.105,0.149 ± 0.106,0.153 ± 0.107,0.156 ± 0.108,NA
6,0.15 ± 0.102,0.154 ± 0.104,0.162 ± 0.112,0.158 ± 0.102,0.148 ± 0.097,0.15 ± 0.098,0.15 ± 0.102,0.163 ± 0.104,0.15 ± 0.098,NA
7,0.156 ± 0.108,0.161 ± 0.102,0.155 ± 0.103,0.156 ± 0.107,0.153 ± 0.104,0.16 ± 0.109,0.157 ± 0.103,0.153 ± 0.105,0.156 ± 0.11,NA
8,0.154 ± 0.098,0.159 ± 0.109,0.159 ± 0.1,0.155 ± 0.101,0.153 ± 0.102,0.154 ± 0.099,0.159 ± 0.103,0.157 ± 0.103,0.151 ± 0.096,NA
9,0.151 ± 0.103,0.157 ± 0.103,0.153 ± 0.098,0.159 ± 0.102,0.158 ± 0.102,0.159 ± 0.108,0.168 ± 0.105,0.151 ± 0.099,0.163 ± 0.109,NA


In [45]:
# group 5, ML + SM 

# read the table compiled in chapter 4. So this cell does not depend on that one to run, it just needs the table file.
df_new = read.table("tables/RF_swapped_set_all_methods_vs_reference.tsv", header=T)

# filter the above table to retain only rf values associated with trees coming from group 1 and method ME + SM 
df_group5_ML_SM = df_new  %>%
  filter(group == "group5_11th", method == "SM_ML")

# Group by the swap_species and swap_seq per swap combination and calculate mean rf_val and sd
df_group5_ML_SM_processed = df_group5_ML_SM %>%
    group_by(swap_species, swap_seq) %>%
    summarise(
        mean_rf = round(mean(rf_val), 3),
        sd_rf = round(sd(rf_val), 3),
        .groups = "drop"
    ) %>%
    # Combine mean and sd into one string
    mutate(mean_sd = paste0(mean_rf, " ± ", sd_rf)) %>%
    select(swap_species, swap_seq, mean_sd) %>%
    # Create a wide format 10x10 matrix/table
    pivot_wider(names_from = swap_seq, values_from = mean_sd, values_fill = NA) %>%
    # Arrange rows in order
    arrange(swap_species)

df_group5_ML_SM_processed
write.table(df_group5_ML_SM_processed, file = "tables/avg_RF_swapped_set_group5_ML_SM_vs_reference.tsv", quote = FALSE, row.names = FALSE, sep = "\t")

swap_species,2,3,4,5,6,7,8,9,10,15
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,0.11 ± 0.084,0.106 ± 0.081,0.107 ± 0.083,0.106 ± 0.088,0.108 ± 0.086,0.111 ± 0.08,0.107 ± 0.082,0.103 ± 0.083,0.108 ± 0.084,NA
2,0.116 ± 0.087,0.104 ± 0.08,0.103 ± 0.086,0.112 ± 0.084,0.104 ± 0.085,0.108 ± 0.085,0.109 ± 0.085,0.108 ± 0.082,0.111 ± 0.085,NA
3,0.107 ± 0.083,0.113 ± 0.09,0.113 ± 0.088,0.104 ± 0.085,0.108 ± 0.086,0.111 ± 0.085,0.111 ± 0.086,0.112 ± 0.09,0.108 ± 0.092,NA
4,0.107 ± 0.087,0.109 ± 0.095,0.11 ± 0.085,0.111 ± 0.085,0.117 ± 0.088,0.113 ± 0.093,0.119 ± 0.094,0.114 ± 0.102,0.112 ± 0.094,NA
5,0.109 ± 0.083,0.113 ± 0.09,0.113 ± 0.092,0.114 ± 0.091,0.111 ± 0.089,0.111 ± 0.089,0.119 ± 0.093,0.118 ± 0.095,0.113 ± 0.09,NA
6,0.111 ± 0.089,0.107 ± 0.08,0.109 ± 0.092,0.11 ± 0.085,0.118 ± 0.096,0.12 ± 0.089,0.12 ± 0.094,0.124 ± 0.096,0.113 ± 0.084,NA
7,0.108 ± 0.091,0.114 ± 0.093,0.117 ± 0.087,0.111 ± 0.089,0.124 ± 0.09,0.108 ± 0.091,0.128 ± 0.091,0.118 ± 0.099,0.129 ± 0.099,NA
8,0.106 ± 0.087,0.114 ± 0.096,0.116 ± 0.082,0.114 ± 0.095,0.123 ± 0.097,0.113 ± 0.095,0.121 ± 0.092,0.118 ± 0.096,0.121 ± 0.091,NA
9,0.113 ± 0.09,0.117 ± 0.089,0.109 ± 0.089,0.12 ± 0.094,0.121 ± 0.094,0.13 ± 0.096,0.13 ± 0.098,0.128 ± 0.097,0.139 ± 0.099,NA


In [46]:
# group 5, ML + ST 

# read the table compiled in chapter 4. So this cell does not depend on that one to run, it just needs the table file.
df_new = read.table("tables/RF_swapped_set_all_methods_vs_reference.tsv", header=T)

# filter the above table to retain only rf values associated with trees coming from group 1 and method ME + SM 
df_group5_ML_ST = df_new  %>%
  filter(group == "group5_11th", method == "ST_ML")

# Group by the swap_species and swap_seq per swap combination and calculate mean rf_val and sd
df_group5_ML_ST_processed = df_group5_ML_ST %>%
    group_by(swap_species, swap_seq) %>%
    summarise(
        mean_rf = round(mean(rf_val), 3),
        sd_rf = round(sd(rf_val), 3),
        .groups = "drop"
    ) %>%
    # Combine mean and sd into one string
    mutate(mean_sd = paste0(mean_rf, " ± ", sd_rf)) %>%
    select(swap_species, swap_seq, mean_sd) %>%
    # Create a wide format 10x10 matrix/table
    pivot_wider(names_from = swap_seq, values_from = mean_sd, values_fill = NA) %>%
    # Arrange rows in order
    arrange(swap_species)

df_group5_ML_ST_processed
write.table(df_group5_ML_ST_processed, file = "tables/avg_RF_swapped_set_group5_ML_ST_vs_reference.tsv", quote = FALSE, row.names = FALSE, sep = "\t")

swap_species,2,3,4,5,6,7,8,9,10,15
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,0.119 ± 0.086,0.115 ± 0.082,0.113 ± 0.084,0.11 ± 0.084,0.114 ± 0.089,0.115 ± 0.086,0.111 ± 0.082,0.112 ± 0.082,0.114 ± 0.083,NA
2,0.115 ± 0.085,0.109 ± 0.081,0.109 ± 0.08,0.113 ± 0.083,0.116 ± 0.084,0.112 ± 0.089,0.115 ± 0.083,0.116 ± 0.088,0.113 ± 0.081,NA
3,0.117 ± 0.087,0.113 ± 0.087,0.111 ± 0.082,0.112 ± 0.085,0.117 ± 0.089,0.113 ± 0.086,0.116 ± 0.083,0.117 ± 0.086,0.114 ± 0.091,NA
4,0.113 ± 0.085,0.114 ± 0.083,0.113 ± 0.082,0.111 ± 0.088,0.115 ± 0.081,0.121 ± 0.09,0.121 ± 0.088,0.117 ± 0.09,0.115 ± 0.086,NA
5,0.112 ± 0.082,0.115 ± 0.084,0.113 ± 0.081,0.113 ± 0.078,0.115 ± 0.084,0.109 ± 0.082,0.113 ± 0.083,0.117 ± 0.085,0.108 ± 0.079,NA
6,0.115 ± 0.086,0.114 ± 0.082,0.113 ± 0.088,0.112 ± 0.081,0.115 ± 0.085,0.112 ± 0.082,0.111 ± 0.087,0.113 ± 0.09,0.105 ± 0.08,NA
7,0.111 ± 0.088,0.112 ± 0.079,0.116 ± 0.084,0.115 ± 0.082,0.111 ± 0.078,0.113 ± 0.088,0.117 ± 0.088,0.112 ± 0.087,0.111 ± 0.087,NA
8,0.113 ± 0.087,0.115 ± 0.085,0.108 ± 0.08,0.11 ± 0.079,0.113 ± 0.082,0.109 ± 0.078,0.112 ± 0.083,0.111 ± 0.086,0.121 ± 0.089,NA
9,0.112 ± 0.086,0.114 ± 0.086,0.116 ± 0.089,0.115 ± 0.089,0.113 ± 0.084,0.114 ± 0.084,0.116 ± 0.088,0.122 ± 0.09,0.124 ± 0.087,NA


In [4]:
# All groups, ML + ST 

# read the table compiled in chapter 4. So this cell does not depend on that one to run, it just needs the table file.
df_new = read.table("tables/RF_swapped_set_all_methods_vs_reference.tsv", header=T)

# filter the above table to retain only rf values associated with trees coming from method ML + ST 
df_ML_ST = df_new  %>%
  filter(method == "ST_ML")

# Group by the swap_species and swap_seq per swap combination and calculate mean rf_val and sd
df_ML_ST_processed = df_ML_ST %>%
    group_by(swap_species, swap_seq) %>%
    summarise(
        mean_rf = round(mean(rf_val), 3),
        sd_rf = round(sd(rf_val), 3),
        .groups = "drop"
    ) %>%
    # Combine mean and sd into one string
    mutate(mean_sd = paste0(mean_rf, " ± ", sd_rf)) %>%
    select(swap_species, swap_seq, mean_sd) %>%
    # Create a wide format 10x10 matrix/table
    pivot_wider(names_from = swap_seq, values_from = mean_sd, values_fill = NA) %>%
    # Arrange rows in order
    arrange(swap_species)

df_ML_ST_processed

swap_species,2,3,4,5,6,7,8,9,10,15
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,0.203 ± 0.144,0.202 ± 0.145,0.202 ± 0.146,0.199 ± 0.143,0.202 ± 0.144,0.201 ± 0.144,0.203 ± 0.146,0.201 ± 0.145,0.203 ± 0.146,NA
2,0.201 ± 0.146,0.201 ± 0.146,0.201 ± 0.145,0.201 ± 0.145,0.202 ± 0.146,0.201 ± 0.143,0.203 ± 0.143,0.203 ± 0.144,0.2 ± 0.143,NA
3,0.202 ± 0.145,0.202 ± 0.146,0.2 ± 0.143,0.204 ± 0.146,0.203 ± 0.147,0.202 ± 0.144,0.204 ± 0.144,0.202 ± 0.144,0.202 ± 0.144,NA
4,0.204 ± 0.145,0.202 ± 0.146,0.202 ± 0.144,0.203 ± 0.145,0.201 ± 0.143,0.204 ± 0.143,0.202 ± 0.144,0.2 ± 0.144,0.201 ± 0.141,NA
5,0.202 ± 0.146,0.201 ± 0.144,0.202 ± 0.146,0.203 ± 0.143,0.201 ± 0.141,0.201 ± 0.144,0.204 ± 0.144,0.205 ± 0.145,0.202 ± 0.144,NA
6,0.204 ± 0.145,0.201 ± 0.142,0.204 ± 0.147,0.204 ± 0.144,0.205 ± 0.145,0.204 ± 0.142,0.205 ± 0.148,0.202 ± 0.143,0.204 ± 0.146,NA
7,0.202 ± 0.147,0.205 ± 0.145,0.203 ± 0.143,0.205 ± 0.147,0.202 ± 0.143,0.207 ± 0.15,0.203 ± 0.147,0.205 ± 0.144,0.204 ± 0.147,NA
8,0.201 ± 0.143,0.203 ± 0.145,0.202 ± 0.144,0.202 ± 0.14,0.208 ± 0.148,0.204 ± 0.145,0.205 ± 0.149,0.202 ± 0.145,0.207 ± 0.149,NA
9,0.203 ± 0.143,0.202 ± 0.142,0.203 ± 0.145,0.204 ± 0.14,0.203 ± 0.142,0.205 ± 0.152,0.205 ± 0.144,0.214 ± 0.148,0.206 ± 0.144,NA


In [10]:
# All groups, ML + SM 

# read the table compiled in chapter 4. So this cell does not depend on that one to run, it just needs the table file.
df_new = read.table("tables/RF_swapped_set_all_methods_vs_reference.tsv", header=T)

# filter the above table to retain only rf values associated with trees coming from method ML + SM 
df_ML_SM = df_new  %>%
  filter(method == "SM_ML")

# Group by the swap_species and swap_seq per swap combination and calculate mean rf_val and sd
df_ML_SM_processed = df_ML_SM %>%
    group_by(swap_species, swap_seq) %>%
    summarise(
        mean_rf = round(mean(rf_val), 3),
        sd_rf = round(sd(rf_val), 3),
        .groups = "drop"
    ) %>%
    # Combine mean and sd into one string
    mutate(mean_sd = paste0(mean_rf, " ± ", sd_rf)) %>%
    select(swap_species, swap_seq, mean_sd) %>%
    # Create a wide format 10x10 matrix/table
    pivot_wider(names_from = swap_seq, values_from = mean_sd, values_fill = NA) %>%
    # Arrange rows in order
    arrange(swap_species)

df_ML_SM_processed

swap_species,2,3,4,5,6,7,8,9,10,15
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,0.19 ± 0.142,0.19 ± 0.14,0.192 ± 0.142,0.191 ± 0.143,0.191 ± 0.141,0.192 ± 0.14,0.19 ± 0.141,0.189 ± 0.14,0.19 ± 0.14,NA
2,0.192 ± 0.142,0.19 ± 0.144,0.189 ± 0.143,0.192 ± 0.144,0.193 ± 0.148,0.191 ± 0.138,0.191 ± 0.142,0.193 ± 0.142,0.195 ± 0.144,NA
3,0.19 ± 0.145,0.191 ± 0.14,0.192 ± 0.143,0.192 ± 0.144,0.192 ± 0.137,0.193 ± 0.145,0.194 ± 0.143,0.191 ± 0.141,0.195 ± 0.141,NA
4,0.189 ± 0.141,0.193 ± 0.144,0.193 ± 0.142,0.192 ± 0.14,0.198 ± 0.14,0.195 ± 0.144,0.194 ± 0.143,0.188 ± 0.143,0.197 ± 0.143,NA
5,0.194 ± 0.144,0.194 ± 0.141,0.195 ± 0.14,0.194 ± 0.142,0.192 ± 0.142,0.191 ± 0.144,0.199 ± 0.141,0.193 ± 0.14,0.193 ± 0.142,NA
6,0.197 ± 0.139,0.197 ± 0.142,0.196 ± 0.139,0.196 ± 0.139,0.198 ± 0.144,0.198 ± 0.145,0.201 ± 0.141,0.201 ± 0.145,0.2 ± 0.145,NA
7,0.196 ± 0.145,0.195 ± 0.142,0.195 ± 0.141,0.194 ± 0.139,0.198 ± 0.139,0.2 ± 0.149,0.199 ± 0.138,0.198 ± 0.143,0.198 ± 0.137,NA
8,0.19 ± 0.141,0.196 ± 0.141,0.198 ± 0.14,0.199 ± 0.146,0.202 ± 0.142,0.206 ± 0.146,0.205 ± 0.147,0.205 ± 0.147,0.206 ± 0.144,NA
9,0.193 ± 0.143,0.199 ± 0.141,0.201 ± 0.147,0.206 ± 0.143,0.203 ± 0.146,0.206 ± 0.149,0.208 ± 0.144,0.21 ± 0.147,0.215 ± 0.148,NA


In [11]:
# All groups, ME + SM 

# read the table compiled in chapter 4. So this cell does not depend on that one to run, it just needs the table file.
df_new = read.table("tables/RF_swapped_set_all_methods_vs_reference.tsv", header=T)

# filter the above table to retain only rf values associated with trees coming from method ME + SM 
df_ME_SM = df_new  %>%
  filter(method == "SM_ME")

# Group by the swap_species and swap_seq per swap combination and calculate mean rf_val and sd
df_ME_SM_processed = df_ME_SM %>%
    group_by(swap_species, swap_seq) %>%
    summarise(
        mean_rf = round(mean(rf_val), 3),
        sd_rf = round(sd(rf_val), 3),
        .groups = "drop"
    ) %>%
    # Combine mean and sd into one string
    mutate(mean_sd = paste0(mean_rf, " ± ", sd_rf)) %>%
    select(swap_species, swap_seq, mean_sd) %>%
    # Create a wide format 10x10 matrix/table
    pivot_wider(names_from = swap_seq, values_from = mean_sd, values_fill = NA) %>%
    # Arrange rows in order
    arrange(swap_species)

df_ME_SM_processed

swap_species,2,3,4,5,6,7,8,9,10,15
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,0.239 ± 0.15,0.239 ± 0.148,0.241 ± 0.146,0.239 ± 0.148,0.241 ± 0.148,0.235 ± 0.147,0.241 ± 0.151,0.239 ± 0.149,0.234 ± 0.15,NA
2,0.24 ± 0.148,0.242 ± 0.149,0.241 ± 0.15,0.236 ± 0.144,0.245 ± 0.151,0.239 ± 0.146,0.242 ± 0.149,0.238 ± 0.148,0.24 ± 0.146,NA
3,0.239 ± 0.148,0.243 ± 0.149,0.244 ± 0.148,0.243 ± 0.148,0.241 ± 0.15,0.238 ± 0.152,0.239 ± 0.152,0.236 ± 0.149,0.238 ± 0.146,NA
4,0.241 ± 0.149,0.241 ± 0.15,0.241 ± 0.147,0.235 ± 0.151,0.242 ± 0.151,0.241 ± 0.149,0.239 ± 0.153,0.235 ± 0.148,0.245 ± 0.149,NA
5,0.237 ± 0.147,0.239 ± 0.152,0.242 ± 0.148,0.24 ± 0.152,0.24 ± 0.152,0.24 ± 0.15,0.24 ± 0.15,0.232 ± 0.144,0.236 ± 0.148,NA
6,0.243 ± 0.148,0.235 ± 0.15,0.241 ± 0.146,0.239 ± 0.15,0.233 ± 0.151,0.24 ± 0.156,0.241 ± 0.151,0.244 ± 0.148,0.244 ± 0.149,NA
7,0.239 ± 0.149,0.24 ± 0.149,0.239 ± 0.148,0.237 ± 0.147,0.241 ± 0.152,0.239 ± 0.15,0.243 ± 0.144,0.239 ± 0.153,0.237 ± 0.147,NA
8,0.24 ± 0.145,0.24 ± 0.149,0.242 ± 0.145,0.248 ± 0.149,0.237 ± 0.153,0.24 ± 0.154,0.239 ± 0.153,0.241 ± 0.15,0.239 ± 0.152,NA
9,0.241 ± 0.152,0.243 ± 0.147,0.241 ± 0.149,0.246 ± 0.144,0.244 ± 0.147,0.246 ± 0.152,0.25 ± 0.152,0.243 ± 0.152,0.243 ± 0.15,NA


In [13]:
# All groups, ME + ST 

# read the table compiled in chapter 4. So this cell does not depend on that one to run, it just needs the table file.
df_new = read.table("tables/RF_swapped_set_all_methods_vs_reference.tsv", header=T)

# filter the above table to retain only rf values associated with trees coming from method ME + ST 
df_ME_ST = df_new  %>%
  filter(method == "ST_ME")

# Group by the swap_species and swap_seq per swap combination and calculate mean rf_val and sd
df_ME_ST_processed = df_ME_ST %>%
    group_by(swap_species, swap_seq) %>%
    summarise(
        mean_rf = round(mean(rf_val), 3),
        sd_rf = round(sd(rf_val), 3),
        .groups = "drop"
    ) %>%
    # Combine mean and sd into one string
    mutate(mean_sd = paste0(mean_rf, " ± ", sd_rf)) %>%
    select(swap_species, swap_seq, mean_sd) %>%
    # Create a wide format 10x10 matrix/table
    pivot_wider(names_from = swap_seq, values_from = mean_sd, values_fill = NA) %>%
    # Arrange rows in order
    arrange(swap_species)

df_ME_ST_processed

swap_species,2,3,4,5,6,7,8,9,10,15
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,0.245 ± 0.156,0.244 ± 0.156,0.244 ± 0.154,0.243 ± 0.156,0.245 ± 0.157,0.244 ± 0.156,0.243 ± 0.155,0.246 ± 0.157,0.244 ± 0.154,NA
2,0.244 ± 0.155,0.244 ± 0.156,0.245 ± 0.155,0.245 ± 0.154,0.247 ± 0.157,0.246 ± 0.155,0.247 ± 0.155,0.245 ± 0.156,0.245 ± 0.156,NA
3,0.244 ± 0.157,0.244 ± 0.155,0.246 ± 0.158,0.247 ± 0.154,0.247 ± 0.156,0.243 ± 0.153,0.243 ± 0.157,0.245 ± 0.156,0.245 ± 0.153,NA
4,0.246 ± 0.157,0.244 ± 0.156,0.245 ± 0.153,0.248 ± 0.155,0.245 ± 0.157,0.247 ± 0.158,0.245 ± 0.152,0.245 ± 0.156,0.246 ± 0.155,NA
5,0.245 ± 0.156,0.243 ± 0.153,0.247 ± 0.156,0.247 ± 0.155,0.245 ± 0.157,0.247 ± 0.157,0.245 ± 0.157,0.245 ± 0.156,0.248 ± 0.153,NA
6,0.244 ± 0.156,0.241 ± 0.153,0.248 ± 0.154,0.249 ± 0.157,0.245 ± 0.155,0.247 ± 0.154,0.247 ± 0.156,0.247 ± 0.154,0.246 ± 0.155,NA
7,0.246 ± 0.155,0.248 ± 0.155,0.246 ± 0.155,0.244 ± 0.157,0.245 ± 0.152,0.249 ± 0.158,0.25 ± 0.154,0.247 ± 0.154,0.245 ± 0.157,NA
8,0.246 ± 0.155,0.249 ± 0.155,0.25 ± 0.154,0.247 ± 0.156,0.247 ± 0.153,0.249 ± 0.154,0.252 ± 0.155,0.253 ± 0.154,0.249 ± 0.152,NA
9,0.244 ± 0.154,0.248 ± 0.154,0.247 ± 0.152,0.248 ± 0.153,0.251 ± 0.154,0.25 ± 0.155,0.251 ± 0.154,0.252 ± 0.156,0.251 ± 0.154,NA
